In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from functions import custom_cv_eval, get_best_features, get_ev_from_df
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
np.set_printoptions(suppress=True)  #Suppress Scientific Notation



In [2]:
df=pd.read_csv("data/owl-with-odds.csv")

In [3]:
#The odds need to be changed to numeric...

subset = ['t1_odds', 't2_odds']
df.dropna(subset=subset ,inplace=True)

df['t1_odds'] = pd.to_numeric(df['t1_odds'], errors='coerce')
df['t2_odds'] = pd.to_numeric(df['t2_odds'], errors='coerce')
df.dropna(subset=subset ,inplace=True)


In [4]:
#Split the validation set
df_train = df[:-60]

df_test = df[-60:]
print(len(df_test))
print(len(df_train))

60
486


In [5]:
#inclusive features
features = ['corona_virus_isolation', 't1_wins_season',
       't1_losses_season', 't2_wins_season', 't2_losses_season',
       't1_matches_season', 't2_matches_season', 't1_win_percent_season',
       't2_win_percent_season', 't1_wins_alltime', 't1_losses_alltime',
       't2_wins_alltime', 't2_losses_alltime', 't1_matches_alltime',
       't2_matches_alltime', 't1_win_percent_alltime',
       't2_win_percent_alltime', 't1_wins_last_3', 't1_losses_last_3',
       't2_wins_last_3', 't2_losses_last_3', 't1_win_percent_last_3',
       't2_win_percent_last_3', 't1_wins_last_5', 't1_losses_last_5',
       't2_wins_last_5', 't2_losses_last_5', 't1_win_percent_last_5',
       't2_win_percent_last_5', 't1_wins_last_10', 't1_losses_last_10',
       't2_wins_last_10', 't2_losses_last_10', 't1_win_percent_last_10',
       't2_win_percent_last_10', 
        't1_wins_vs_t2', 't1_losses_vs_t2',
       't1_matches_vs_t2', 't1_odds', 't2_odds', 'winner_label']

In [6]:
#Full list of possible features

pos_features = ['corona_virus_isolation', 't1_wins_season',
       't1_losses_season', 't2_wins_season', 't2_losses_season',
       't1_matches_season', 't2_matches_season', 't1_win_percent_season',
       't2_win_percent_season', 't1_wins_alltime', 't1_losses_alltime',
       't2_wins_alltime', 't2_losses_alltime', 't1_matches_alltime',
       't2_matches_alltime', 't1_win_percent_alltime',
       't2_win_percent_alltime', 't1_wins_last_3', 't1_losses_last_3',
       't2_wins_last_3', 't2_losses_last_3', 't1_win_percent_last_3',
       't2_win_percent_last_3', 't1_wins_last_5', 't1_losses_last_5',
       't2_wins_last_5', 't2_losses_last_5', 't1_win_percent_last_5',
       't2_win_percent_last_5', 't1_wins_last_10', 't1_losses_last_10',
       't2_wins_last_10', 't2_losses_last_10', 't1_win_percent_last_10',
       't2_win_percent_last_10', 
        't1_wins_vs_t2', 't1_losses_vs_t2',
       't1_matches_vs_t2']

In [7]:
df_test_filtered = df_test[features].copy()
df_train_filtered = df_train[features].copy()
df_test_filtered.dropna(inplace=True)
df_train_filtered.dropna(inplace=True)

In [8]:
model_3 = RandomForestClassifier(random_state=75)
features_3 = ['t2_wins_last_5', 't1_win_percent_season', 't1_losses_season', 't2_losses_last_5', 't1_wins_vs_t2',
            'winner_label', 't1_odds', 't2_odds']
df_3 = df_train_filtered[features_3]

In [9]:
print(df_3.shape)

(389, 8)


In [10]:
print(f"total: {custom_cv_eval(df_3, model_3)}")

total: 117.34041809988898


In [11]:
model_5 = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=12, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=75, splitter='best')
features_5 = ['t1_wins_last_3', 't2_matches_alltime', 'winner_label', 't1_odds', 't2_odds']
df_5 = df_train_filtered[features_5]

In [12]:
print(f"total: {custom_cv_eval(df_5, model_5)}")

total: 2.69412062696951


In [13]:
X=df_train_filtered[features_3].copy()
X_test=df_test_filtered[features_3].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model_3.fit(X, y)
probs=model_3.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)
probs_3 = probs


          Number of matches: 60
          Number of bets: 54
          Number of winning bets: 24
          Number of losing bets: 30
          Number of underdog bets: 26
          Number of underdog wins: 7
          Number of underdog losses: 19
          Number of Favorite bets: 28
          Number of favorite wins: 17
          Number of favorite losses: 11
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -963.3016424729295
          Profit per bet: -17.83891930505425
          Profit per match: -16.055027374548825
          
          


In [14]:
X=df_train_filtered[features_5].copy()
X_test=df_test_filtered[features_5].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model_5.fit(X, y)
probs=model_5.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)
probs_5 = probs


          Number of matches: 60
          Number of bets: 52
          Number of winning bets: 23
          Number of losing bets: 29
          Number of underdog bets: 24
          Number of underdog wins: 6
          Number of underdog losses: 18
          Number of Favorite bets: 28
          Number of favorite wins: 17
          Number of favorite losses: 11
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -1267.6770208695882
          Profit per bet: -24.37840424749208
          Profit per match: -21.12795034782647
          
          


In [15]:
print(probs_5)

[[0.75       0.25      ]
 [0.4        0.6       ]
 [0.57142857 0.42857143]
 [1.         0.        ]
 [0.25       0.75      ]
 [0.5        0.5       ]
 [0.75       0.25      ]
 [1.         0.        ]
 [0.16666667 0.83333333]
 [0.         1.        ]
 [0.57142857 0.42857143]
 [0.5        0.5       ]
 [1.         0.        ]
 [1.         0.        ]
 [0.         1.        ]
 [0.25       0.75      ]
 [1.         0.        ]
 [0.16666667 0.83333333]
 [0.         1.        ]
 [0.4        0.6       ]
 [0.         1.        ]
 [0.4        0.6       ]
 [0.         1.        ]
 [0.5        0.5       ]
 [0.5        0.5       ]
 [1.         0.        ]
 [0.14285714 0.85714286]
 [0.         1.        ]
 [1.         0.        ]
 [0.         1.        ]
 [0.25       0.75      ]
 [1.         0.        ]
 [0.         1.        ]
 [1.         0.        ]
 [1.         0.        ]
 [0.4        0.6       ]
 [0.33333333 0.66666667]
 [0.5        0.5       ]
 [0.         1.        ]
 [1.         0.        ]


### #OK THOSE SUCK.  Let's make some new ones.
#Let's start from scratch with a random forest

In [16]:
model = RandomForestClassifier(random_state=75)

In [20]:
current_features = ['winner_label', 't1_odds', 't2_odds']

In [21]:
f_rf=get_best_features(pos_features, model,  df_train_filtered, current_features)

Current best score is: -11.263813305589146
The best feature was t2_wins_season.  It scored 55.96449534761742
Current best score is: 55.96449534761742
The best feature was t2_matches_alltime.  It scored 67.98979087453237
Current best score is: 67.98979087453237
The best feature was t2_wins_last_5.  It scored 86.76923041322587
Current best score is: 86.76923041322587
The best feature was t1_losses_season.  It scored 114.285777142796
Current best score is: 114.285777142796
NO IMPROVEMENT
FINAL BEST SCORE: 114.285777142796


In [22]:
print(f_rf)

['t1_losses_season', 't2_wins_last_5', 't2_matches_alltime', 't2_wins_season', 'winner_label', 't1_odds', 't2_odds']


In [23]:
n_estimators_pos=[10,50,100,150]
criterion_pos=['gini', 'entropy']
max_depth_pos=[None, 2,4,6,8,10]
min_samples_leaf_pos=[1,2,3,4,5,6]


In [24]:
best_score=0
best_features=0
for ne in n_estimators_pos:
    for c in criterion_pos:
        for md in max_depth_pos:
            for msl in min_samples_leaf_pos:
                model = RandomForestClassifier(random_state=75, n_estimators=ne, criterion=c, max_depth=md,
                                              min_samples_leaf=msl)
                possible_score=custom_cv_eval(df_train_filtered[f_rf], model)
                if possible_score > best_score:
                    best_score=possible_score
                    best_features=[ne, c, md, msl]
                    best_model = model
                    print(f"{best_score} {best_features}")  

51.69191322055357 [10, 'gini', None, 1]
51.78891283360663 [10, 'gini', None, 5]
54.140479879060514 [10, 'gini', 6, 2]
77.5607542723209 [10, 'gini', 6, 5]
89.98489202033515 [10, 'entropy', 8, 5]
129.62060572024484 [50, 'gini', 4, 1]


In [25]:
print(best_model)
print(f_rf)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=75, verbose=0,
                       warm_start=False)
['t1_losses_season', 't2_wins_last_5', 't2_matches_alltime', 't2_wins_season', 'winner_label', 't1_odds', 't2_odds']


In [26]:
X=df_train_filtered[f_rf].copy()
X_test=df_test_filtered[f_rf].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
best_model.fit(X, y)
probs=best_model.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)



          Number of matches: 60
          Number of bets: 38
          Number of winning bets: 16
          Number of losing bets: 22
          Number of underdog bets: 17
          Number of underdog wins: 5
          Number of underdog losses: 12
          Number of Favorite bets: 21
          Number of favorite wins: 11
          Number of favorite losses: 10
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -551.4184081787079
          Profit per bet: -14.511010741544943
          Profit per match: -9.190306802978464
          
          


-14.511010741544943

In [27]:
model = DecisionTreeClassifier(random_state=75)


In [29]:
f_dt=get_best_features(pos_features, model,  df_train_filtered, current_features)

Current best score is: -20.849218958250468
The best feature was t1_matches_alltime.  It scored 41.52321180802822
Current best score is: 41.52321180802822
The best feature was t1_wins_vs_t2.  It scored 42.5031397812096
Current best score is: 42.5031397812096
The best feature was t1_win_percent_season.  It scored 54.99088559717145
Current best score is: 54.99088559717145
The best feature was corona_virus_isolation.  It scored 62.6443369501354
Current best score is: 62.6443369501354
The best feature was t1_win_percent_alltime.  It scored 72.32611056535995
Current best score is: 72.32611056535995
NO IMPROVEMENT
FINAL BEST SCORE: 72.32611056535995


In [30]:
criterion_pos = ['gini', 'entropy']
max_depth_pos = [3, 6, 9, 12, None, 10]
min_samples_leaf_pos = [1,2,3,4,5]


In [31]:
best_score = 0
best_features = []
for c in criterion_pos:
    for md in max_depth_pos:
        for msl in min_samples_leaf_pos:
            model = DecisionTreeClassifier(max_depth=md, min_samples_leaf=msl, criterion=c, random_state=75)
            possible_score=custom_cv_eval(df_train_filtered[f_dt], model)
            if possible_score > best_score:
                best_score=possible_score
                best_features=[c, md, msl]
                best_model = model
                print(f"{best_score} {best_features}")


6.163906275382111 ['gini', 3, 1]
57.63167774830849 ['gini', 9, 1]
61.09941744164337 ['gini', 12, 1]
72.32611056535995 ['gini', None, 1]


In [32]:
print(best_model)
print(f_dt)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=75, splitter='best')
['t1_win_percent_alltime', 'corona_virus_isolation', 't1_win_percent_season', 't1_wins_vs_t2', 't1_matches_alltime', 'winner_label', 't1_odds', 't2_odds']


In [33]:
X=df_train_filtered[f_dt].copy()
X_test=df_test_filtered[f_dt].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
best_model.fit(X, y)
probs=best_model.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)


          Number of matches: 60
          Number of bets: 60
          Number of winning bets: 37
          Number of losing bets: 23
          Number of underdog bets: 16
          Number of underdog wins: 6
          Number of underdog losses: 10
          Number of Favorite bets: 44
          Number of favorite wins: 31
          Number of favorite losses: 13
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -236.97780052463577
          Profit per bet: -3.9496300087439296
          Profit per match: -3.9496300087439296
          
          


-3.9496300087439296

### MODEL 1

In [34]:
model = DecisionTreeClassifier(max_depth=10, min_samples_leaf=3, random_state=75)
features = ['t1_wins_last_3', 't2_matches_alltime', 'winner_label', 't1_odds', 't2_odds']

In [35]:
X=df_train_filtered[features].copy()
X_test=df_test_filtered[features].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model.fit(X, y)
probs=model.predict_proba(X_test)
probs_1 = probs
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)


          Number of matches: 60
          Number of bets: 60
          Number of winning bets: 32
          Number of losing bets: 28
          Number of underdog bets: 23
          Number of underdog wins: 7
          Number of underdog losses: 16
          Number of Favorite bets: 37
          Number of favorite wins: 25
          Number of favorite losses: 12
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -707.3538644761904
          Profit per bet: -11.789231074603174
          Profit per match: -11.789231074603174
          
          


-11.789231074603174

### MODEL 2

In [36]:
model = LogisticRegression(random_state=75, max_iter=1000)
features = ['t2_win_percent_last_10', 't1_losses_alltime', 'winner_label', 't1_odds', 't2_odds']


In [37]:
print(model)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=75, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


model

In [38]:
#SCALER ONLY NEEDED FOR LR

##


X=df_train_filtered[features].copy()


X_test=df_test_filtered[features].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
X_test_list = np.array(X_test)
#SCALER ONLY NEEDED FOR LR
sc=StandardScaler()
X = sc.fit_transform(X)
X_test = sc.transform(X_test)
##



model.fit(X, y)
probs=model.predict_proba(X_test)
probs_2=probs
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)


          Number of matches: 60
          Number of bets: 49
          Number of winning bets: 15
          Number of losing bets: 34
          Number of underdog bets: 42
          Number of underdog wins: 11
          Number of underdog losses: 31
          Number of Favorite bets: 7
          Number of favorite wins: 4
          Number of favorite losses: 3
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -642.5809345637672
          Profit per bet: -13.113896623750351
          Profit per match: -10.709682242729453
          
          


-13.113896623750351

### MODEL 3

In [39]:
model = RandomForestClassifier(random_state=75)
features = ['t2_wins_last_5', 't1_win_percent_season', 't1_losses_season', 't2_losses_last_5', 't1_wins_vs_t2',
            'winner_label', 't1_odds', 't2_odds']

In [40]:
X=df_train_filtered[features].copy()
X_test=df_test_filtered[features].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model.fit(X, y)
probs=model.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)


          Number of matches: 60
          Number of bets: 54
          Number of winning bets: 24
          Number of losing bets: 30
          Number of underdog bets: 26
          Number of underdog wins: 7
          Number of underdog losses: 19
          Number of Favorite bets: 28
          Number of favorite wins: 17
          Number of favorite losses: 11
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -963.3016424729295
          Profit per bet: -17.83891930505425
          Profit per match: -16.055027374548825
          
          


-17.83891930505425

### MODEL 4

In [41]:
model=MLPClassifier(random_state=75) 
features = ['t1_matches_season', 'winner_label', 't1_odds', 't2_odds']

In [42]:
X=df_train_filtered[features].copy()
X_test=df_test_filtered[features].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model.fit(X, y)
probs=model.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)


          Number of matches: 60
          Number of bets: 58
          Number of winning bets: 27
          Number of losing bets: 31
          Number of underdog bets: 23
          Number of underdog wins: 5
          Number of underdog losses: 18
          Number of Favorite bets: 35
          Number of favorite wins: 22
          Number of favorite losses: 13
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -1889.3424355841478
          Profit per bet: -32.57486957903703
          Profit per match: -31.48904059306913
          
          


-32.57486957903703

### MODEL 5

In [43]:
model=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=12, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=75, splitter='best')
features = ['t1_wins_last_3', 't2_matches_alltime', 'winner_label', 't1_odds', 't2_odds']

In [44]:
X=df_train_filtered[features].copy()
X_test=df_test_filtered[features].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model.fit(X, y)
probs=model.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)


          Number of matches: 60
          Number of bets: 52
          Number of winning bets: 23
          Number of losing bets: 29
          Number of underdog bets: 24
          Number of underdog wins: 6
          Number of underdog losses: 18
          Number of Favorite bets: 28
          Number of favorite wins: 17
          Number of favorite losses: 11
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -1267.6770208695882
          Profit per bet: -24.37840424749208
          Profit per match: -21.12795034782647
          
          


-24.37840424749208

### MODEL 6

In [45]:
model=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=6, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=75, verbose=0,
                       warm_start=False)

features = ['t2_wins_last_5', 't1_win_percent_season', 't1_losses_season', 't2_losses_last_5', 't1_wins_vs_t2', 'winner_label', 't1_odds', 't2_odds']


In [46]:
X=df_train_filtered[features].copy()
X_test=df_test_filtered[features].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model.fit(X, y)
probs=model.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)


          Number of matches: 60
          Number of bets: 52
          Number of winning bets: 21
          Number of losing bets: 31
          Number of underdog bets: 28
          Number of underdog wins: 7
          Number of underdog losses: 21
          Number of Favorite bets: 24
          Number of favorite wins: 14
          Number of favorite losses: 10
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -908.2604830078825
          Profit per bet: -17.466547750151587
          Profit per match: -15.137674716798042
          
          


-17.466547750151587

### MODEL 7

In [47]:
model=GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.2, loss='deviance', max_depth=16,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=6, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=75, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

features =['corona_virus_isolation', 't1_wins_season', 't1_losses_season', 't2_wins_season', 't2_losses_season', 't1_matches_season', 't2_matches_season', 't1_win_percent_season', 't2_win_percent_season', 't1_wins_alltime', 't1_losses_alltime', 't2_wins_alltime', 't2_losses_alltime', 't1_matches_alltime', 't2_matches_alltime', 't1_win_percent_alltime', 't2_win_percent_alltime', 't1_wins_last_3', 't1_losses_last_3', 't2_wins_last_3', 't2_losses_last_3', 't1_win_percent_last_3', 't2_win_percent_last_3', 't1_wins_last_5', 't1_losses_last_5', 't2_wins_last_5', 't2_losses_last_5', 't1_win_percent_last_5', 't2_win_percent_last_5', 't1_wins_last_10', 't1_losses_last_10', 't2_wins_last_10', 't2_losses_last_10', 't1_win_percent_last_10', 't2_win_percent_last_10', 't1_wins_vs_t2', 't1_losses_vs_t2', 't1_matches_vs_t2', 'winner_label', 't1_odds', 't2_odds']


In [48]:
X=df_train_filtered[features].copy()
X_test=df_test_filtered[features].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model.fit(X, y)
probs=model.predict_proba(X_test)
probs_7=probs
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)


          Number of matches: 60
          Number of bets: 60
          Number of winning bets: 35
          Number of losing bets: 25
          Number of underdog bets: 14
          Number of underdog wins: 4
          Number of underdog losses: 10
          Number of Favorite bets: 46
          Number of favorite wins: 31
          Number of favorite losses: 15
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -847.7502781849504
          Profit per bet: -14.129171303082506
          Profit per match: -14.129171303082506
          
          


-14.129171303082506

In [49]:
probs_2

array([[0.27610539, 0.72389461],
       [0.54607136, 0.45392864],
       [0.4951614 , 0.5048386 ],
       [0.85051513, 0.14948487],
       [0.33822625, 0.66177375],
       [0.62486496, 0.37513504],
       [0.36920528, 0.63079472],
       [0.73904478, 0.26095522],
       [0.03910704, 0.96089296],
       [0.34300503, 0.65699497],
       [0.46039917, 0.53960083],
       [0.66314031, 0.33685969],
       [0.51804212, 0.48195788],
       [0.66867317, 0.33132683],
       [0.2711652 , 0.7288348 ],
       [0.3545319 , 0.6454681 ],
       [0.57523073, 0.42476927],
       [0.1310039 , 0.8689961 ],
       [0.32425872, 0.67574128],
       [0.54990968, 0.45009032],
       [0.17206136, 0.82793864],
       [0.38536866, 0.61463134],
       [0.40993573, 0.59006427],
       [0.59465472, 0.40534528],
       [0.3068234 , 0.6931766 ],
       [0.35268315, 0.64731685],
       [0.51945784, 0.48054216],
       [0.31363934, 0.68636066],
       [0.53294434, 0.46705566],
       [0.26682848, 0.73317152],
       [0.

In [50]:
probs_7

array([[0.02653916, 0.97346084],
       [0.94746141, 0.05253859],
       [0.84887797, 0.15112203],
       [0.99914325, 0.00085675],
       [0.03792129, 0.96207871],
       [0.99977956, 0.00022044],
       [0.00138298, 0.99861702],
       [0.94534046, 0.05465954],
       [0.00067018, 0.99932982],
       [0.05055432, 0.94944568],
       [0.30841925, 0.69158075],
       [0.99940374, 0.00059626],
       [0.99603612, 0.00396388],
       [0.9978515 , 0.0021485 ],
       [0.00004862, 0.99995138],
       [0.04428489, 0.95571511],
       [0.99591711, 0.00408289],
       [0.00017825, 0.99982175],
       [0.22344479, 0.77655521],
       [0.89242808, 0.10757192],
       [0.00027082, 0.99972918],
       [0.58508395, 0.41491605],
       [0.000372  , 0.999628  ],
       [0.94978939, 0.05021061],
       [0.00003914, 0.99996086],
       [0.51922747, 0.48077253],
       [0.942345  , 0.057655  ],
       [0.00257369, 0.99742631],
       [0.39832041, 0.60167959],
       [0.00005992, 0.99994008],
       [0.

In [51]:
ensemble_probs=(probs_2 + probs_1) * .5

In [52]:
print(ensemble_probs)

[[0.13805269 0.86194731]
 [0.47303568 0.52696432]
 [0.41424736 0.58575264]
 [0.92525757 0.07474243]
 [0.16911312 0.83088688]
 [0.51243248 0.48756752]
 [0.18460264 0.81539736]
 [0.86952239 0.13047761]
 [0.14455352 0.85544648]
 [0.17150252 0.82849748]
 [0.73019958 0.26980042]
 [0.53157016 0.46842984]
 [0.75902106 0.24097894]
 [0.83433659 0.16566341]
 [0.1355826  0.8644174 ]
 [0.47726595 0.52273405]
 [0.78761536 0.21238464]
 [0.23216862 0.76783138]
 [0.16212936 0.83787064]
 [0.47495484 0.52504516]
 [0.08603068 0.91396932]
 [0.39268433 0.60731567]
 [0.20496786 0.79503214]
 [0.79732736 0.20267264]
 [0.4034117  0.5965883 ]
 [0.67634158 0.32365842]
 [0.33115749 0.66884251]
 [0.15681967 0.84318033]
 [0.76647217 0.23352783]
 [0.13341424 0.86658576]
 [0.50239595 0.49760405]
 [0.77164419 0.22835581]
 [0.25193914 0.74806086]
 [0.81632318 0.18367682]
 [0.72499708 0.27500292]
 [0.44739891 0.55260109]
 [0.59666667 0.40333333]
 [0.35522978 0.64477022]
 [0.17252843 0.82747157]
 [0.72333212 0.27666788]


In [53]:
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], ensemble_probs[:, 0], ensemble_probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)


          Number of matches: 60
          Number of bets: 52
          Number of winning bets: 23
          Number of losing bets: 29
          Number of underdog bets: 26
          Number of underdog wins: 7
          Number of underdog losses: 19
          Number of Favorite bets: 26
          Number of favorite wins: 16
          Number of favorite losses: 10
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -996.0882092553933
          Profit per bet: -19.15554248568064
          Profit per match: -16.601470154256553
          
          


-19.15554248568064

### MODEL X

In [54]:
model = LogisticRegression()

In [55]:
X=df_train_filtered.copy()


X_test=df_test_filtered.copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
X_test_list = np.array(X_test)
#SCALER ONLY NEEDED FOR LR
sc=StandardScaler()
X = sc.fit_transform(X)
X_test = sc.transform(X_test)
##



model.fit(X, y)
probs=model.predict_proba(X_test)
probs_2=probs
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)


          Number of matches: 60
          Number of bets: 49
          Number of winning bets: 15
          Number of losing bets: 34
          Number of underdog bets: 46
          Number of underdog wins: 13
          Number of underdog losses: 33
          Number of Favorite bets: 3
          Number of favorite wins: 2
          Number of favorite losses: 1
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -717.978021978022
          Profit per bet: -14.65261269342902
          Profit per match: -11.966300366300366
          
          


-14.65261269342902